**General variables**

In [ ]:
$servers = Get-DbaRegServer -SqlInstance vmpfrancia | Select-Object -Unique -ExpandProperty ServerName
$listener = Get-DbaRegServer -SqlInstance vmpfrancia -Group "Always On AG\Listener" | Select-Object -Unique -ExpandProperty ServerName

$folderName = "\DbatoolsExport\HourlyReport"
$Documents = [Environment]::GetFolderPath("MyDocuments")
$DestFolder = $Documents+$folderName

if (!(Test-Path $DestFolder)) {
    New-Item -itemType Directory -Path $Documents -Name $FolderName
}

**CPU Usage**

In [ ]:
$cpu_ringbuffer = Get-DbaCpuRingBuffer -SqlInstance $servers -CollectionMinutes 240 | Select-Object * | ConvertTo-DbaDataTable
$cpu_ringbuffer | Export-Csv -Path $DestFolder"\cpu_ringbuffer.csv" -NoTypeInformation

**Memory Status**

In [ ]:
#$memoryStatus = Get-DbaDbccMemoryStatus -SqlInstance localhost | Where-Object {$_.Type -eq 'Process/System Counts'} | ConvertTo-DbaDataTable
$query = 'select @@SERVERNAME, (physical_memory_in_use_kb/1024) Phy_Memory_usedby_Sqlserver_MB, (locked_page_allocations_kb/1024) Locked_pages_used_Sqlserver_MB, (virtual_address_space_committed_kb/1024) Total_Memory_UsedBySQLServer_MB, process_physical_memory_low, process_virtual_memory_low from sys.dm_os_process_memory'
$memoryStatus = Invoke-DbaQuery -SqlInstance $servers -Query $query | ConvertTo-DbaDataTable
$memoryStatus | Export-Csv -Path $DestFolder"\memory_status.csv" -NoTypeInformation

**Disk Performance**

In [ ]:
$DiskSpeed = Test-DbaDiskSpeed -SqlInstance $servers -AggregateBy 'Database' | Select-Object * | ConvertTo-DbaDataTable
$DiskSpeed | Export-Csv -Path $DestFolder"\disk_speed.csv" -NoTypeInformation

**Availability Groups**

In [ ]:
$AGState = Get-DbaAgReplica -SqlInstance $listener | Select-Object -Unique SqlInstance, AvailabilityGroup, Replica, ConnectionState, AvailabilityMode, FailoverMode, Role | ConvertTo-DbaDataTable
$AGState | Export-Csv -Path $DestFolder"\ag_state.csv" -NoTypeInformation

**Mirroring**

In [ ]:
$Mirroring = Get-DbaRegisteredServer -SqlInstance vmpfrancia -Group MIRROR | Get-DbaDbMirror | Select-Object SqlInstance, Name, MirroringStatus, MirroringPartnerInstance | ConvertTo-DbaDataTable
$Mirroring | Export-Csv -Path $DestFolder"\mirroring.csv" -NoTypeInformation

**Suspect Pages**

In [ ]:
$SuspectPages = Get-DbaSuspectPage -SqlInstance $servers
$SuspectPages | Export-Csv -Path $DestFolder"\suspect_pages.csv" -NoTypeInformation